In [226]:
import pandas as pd
from path import Path

import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit
import time
import datetime
import copy

import numpy as np
from sklearn import preprocessing
%matplotlib inline

In [3]:
track_cmc_merged_df = pd.read_csv(Path(f"../../../../data/cleandata/track_cmc_merged_df.csv"))

In [19]:
track_cmc_merged_df.iloc[704]

rank_cnc                               1380
Name                                 UChain
Ticker                                  UCN
market_cap                          155,884
price                              0.000511
volume24                             50,389
pct1h                                     1
pct24h                                -0.66
pct7d                                 -0.76
country                           Singapore
platform                           Ethereum
pre_ico_end             2017-09-28 03:00:00
pre_ico_start           2017-09-28 03:00:00
rating                                    5
status                               Closed
Name_y                          VerifyUnion
type                                    ICO
End                     2017-12-03 00:00:00
Start                   2017-10-03 00:00:00
Duration         61 days 00:00:00.000000000
pre_Duration      0 days 00:00:00.000000000
Name: 704, dtype: object

In [49]:
ticker_sentiments_df = pd.read_csv(Path(f"../../../../data/cleandata/ticker_sentiments.csv"))

In [50]:
ticker_sentiments_df.rename(columns = {'ticker':'Ticker'}, inplace = True)

In [134]:
merged_inner = pd.merge(left=track_cmc_merged_df,right=ticker_sentiments_df, left_on='Ticker', right_on='Ticker')
merged_inner.head(12)

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,type,End,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts
0,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,ICO,2017-07-21 07:00:00,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100
1,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,ICO,2018-06-12 00:00:00,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100
2,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,Switzerland,...,ICO,2017-01-01 00:00:00,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93
3,18,Tezos,XTZ,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,United States of America,...,ICO,2017-07-14 12:00:00,2017-07-01 12:00:00,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49
4,19,Cosmos,ATOM,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,Canada,...,ICO,2017-04-06 13:00:00,2017-04-06 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67
5,29,Basic Attenti...,BAT,"347,237,523",0.256002,"73,875,875",-1.56,-1.64,1.16,United States of America,...,ICO,2017-05-31 23:00:00,2017-05-31 00:00:00,0 days 23:00:00.000000000,0 days 00:00:00.000000000,0.830295,0.031600,0.837910,0.130530,100
6,34,Qtum,QTUM,"224,670,479",2.34,"348,323,741",-0.52,0.08,5.38,Singapore,...,ICO,2017-03-21 12:00:00,2017-03-16 12:00:00,5 days 00:00:00.000000000,0 days 00:00:00.000000000,0.678774,0.021088,0.913559,0.065382,34
7,36,0x,ZRX,"171,969,043",0.285660,"20,011,107",-1.93,-5.29,-7.94,United States of America,...,ICO,2017-08-16 17:00:00,2017-08-15 00:00:00,1 days 17:00:00.000000000,0 days 00:00:00.000000000,0.729420,0.017843,0.904098,0.078078,51
8,42,OmiseGO,OMG,"141,224,124",1.01,"90,308,960",-0.97,2.19,1.07,Singapore,...,ICO,2017-06-23 13:00:00,2017-06-23 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.772240,0.038130,0.855610,0.106210,100
9,47,Augur,REP,"122,772,944",11.16,"9,187,231",-1.84,-3.80,-1.21,United States of America,...,ICO,2015-10-01 00:00:00,2015-08-17 00:00:00,45 days 00:00:00.000000000,0 days 00:00:00.000000000,0.645049,0.029615,0.883738,0.086723,65


In [137]:
merged_inner.drop(merged_inner.loc[merged_inner['Ticker']=='ALGO'].index, inplace = True)

In [144]:
merged_inner.drop(merged_inner.loc[merged_inner['Ticker']=='AIB'].index, inplace = True)

In [191]:
merged_inner.reset_index(inplace=True)

In [196]:
merged_inner.drop(columns = ['index'], inplace = True)

In [192]:
merged_inner.shape

(362, 29)

In [194]:
merged_inner.head(12)

,index,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,...,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,custom_index,Longevity
0,0,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,...,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100,17867.360000,"848 days, 21:41:57.998914"
1,1,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,...,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100,1785.240000,"522 days, 21:41:58.004910"
2,2,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,...,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93,44.744046,"1049 days, 21:41:58.005911"
3,3,18,Tezos,XTZ,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,...,2017-07-01 12:00:00,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49,1060.200000,"855 days, 21:41:58.006910"
4,4,19,Cosmos,ATOM,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,...,2017-04-06 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67,3806.460000,"954 days, 21:41:58.007909"
5,5,29,Basic Attenti...,BAT,"347,237,523",0.256002,"73,875,875",-1.56,-1.64,1.16,...,2017-05-31 00:00:00,0 days 23:00:00.000000000,0 days 00:00:00.000000000,0.830295,0.031600,0.837910,0.130530,100,230.145798,"899 days, 21:41:58.008908"
6,6,34,Qtum,QTUM,"224,670,479",2.34,"348,323,741",-0.52,0.08,5.38,...,2017-03-16 12:00:00,5 days 00:00:00.000000000,0 days 00:00:00.000000000,0.678774,0.021088,0.913559,0.065382,34,2269.800000,"970 days, 21:41:58.010907"
7,7,36,0x,ZRX,"171,969,043",0.285660,"20,011,107",-1.93,-5.29,-7.94,...,2017-08-15 00:00:00,1 days 17:00:00.000000000,0 days 00:00:00.000000000,0.729420,0.017843,0.904098,0.078078,51,234.812520,"822 days, 21:41:58.011907"
8,8,42,OmiseGO,OMG,"141,224,124",1.01,"90,308,960",-0.97,2.19,1.07,...,2017-06-23 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.772240,0.038130,0.855610,0.106210,100,884.760000,"876 days, 21:41:58.012906"
9,9,47,Augur,REP,"122,772,944",11.16,"9,187,231",-1.84,-3.80,-1.21,...,2015-08-17 00:00:00,45 days 00:00:00.000000000,0 days 00:00:00.000000000,0.645049,0.029615,0.883738,0.086723,65,16818.120000,"1507 days, 21:41:58.013905"


In [166]:
merged_inner['Longevity'] = 0
merged_inner['custom_index'] = 0

In [197]:
def custom_index_success(df):
    
    for i in range(len(df)) : 
        # get the longevity of the ICO
        longevity = datetime.datetime.now() - datetime.datetime.strptime(df.loc[i, "End"][:10], '%Y-%m-%d')
        # return to a column for df
        df.loc[i, "Longevity"] = longevity
        # get current price of df, remove commas
        current_price = df.loc[i, "price"].replace(',', '')
        # create custom index
        custom = float(current_price) / (1/(float(str(longevity)[:4])))
        # retrun to df
        df.loc[i, "custom_index"] = custom
    return df

In [199]:
merged_df = custom_index_success(merged_inner)
merged_df

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,custom_index,Longevity
0,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100,17867.360000,"848 days, 21:47:15.620106"
1,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100,1785.240000,"522 days, 21:47:15.622092"
2,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,Switzerland,...,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93,44.744046,"1049 days, 21:47:15.623091"
3,18,Tezos,XTZ,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,United States of America,...,2017-07-01 12:00:00,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49,1060.200000,"855 days, 21:47:15.624100"
4,19,Cosmos,ATOM,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,Canada,...,2017-04-06 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67,3806.460000,"954 days, 21:47:15.624100"
5,29,Basic Attenti...,BAT,"347,237,523",0.256002,"73,875,875",-1.56,-1.64,1.16,United States of America,...,2017-05-31 00:00:00,0 days 23:00:00.000000000,0 days 00:00:00.000000000,0.830295,0.031600,0.837910,0.130530,100,230.145798,"899 days, 21:47:15.626090"
6,34,Qtum,QTUM,"224,670,479",2.34,"348,323,741",-0.52,0.08,5.38,Singapore,...,2017-03-16 12:00:00,5 days 00:00:00.000000000,0 days 00:00:00.000000000,0.678774,0.021088,0.913559,0.065382,34,2269.800000,"970 days, 21:47:15.627090"
7,36,0x,ZRX,"171,969,043",0.285660,"20,011,107",-1.93,-5.29,-7.94,United States of America,...,2017-08-15 00:00:00,1 days 17:00:00.000000000,0 days 00:00:00.000000000,0.729420,0.017843,0.904098,0.078078,51,234.812520,"822 days, 21:47:15.628088"
8,42,OmiseGO,OMG,"141,224,124",1.01,"90,308,960",-0.97,2.19,1.07,Singapore,...,2017-06-23 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.772240,0.038130,0.855610,0.106210,100,884.760000,"876 days, 21:47:15.629088"
9,47,Augur,REP,"122,772,944",11.16,"9,187,231",-1.84,-3.80,-1.21,United States of America,...,2015-08-17 00:00:00,45 days 00:00:00.000000000,0 days 00:00:00.000000000,0.645049,0.029615,0.883738,0.086723,65,16818.120000,"1507 days, 21:47:15.630087"


# using min max scaler

In [292]:
#min max scaler not doing much as the min value is practically 0... thus its taking all the values and dividing it by the max value
list = (merged_df['custom_index'])/(merged_df['custom_index'].max())


In [264]:
merged_df['custom_index'].min()

5.4631200000000006e-05

In [273]:
scaler = preprocessing.MinMaxScaler()

In [274]:
merged_df['custom_index_scaled'] = scaler.fit_transform(merged_df[['custom_index']])

In [275]:
merged_df['custom_index_scaled']

0      1.595565e-02
1      1.594229e-03
2      3.995664e-05
3      9.467646e-04
4      3.399190e-03
5      2.055215e-04
6      2.026944e-03
7      2.096889e-04
8      7.900957e-04
9      1.501868e-02
10     8.811699e-04
11     5.390470e-04
12     1.985679e-04
13     2.799969e-07
14     8.677370e-04
15     2.573422e-07
16     1.849497e-04
17     3.367239e-03
18     6.078021e-04
19     1.014646e-05
20     4.453772e-05
21     6.380371e-05
22     1.133010e-03
23     2.146207e-05
24     4.367032e-04
25     1.256529e-04
26     5.422387e-06
27     3.086726e-05
28     4.122356e-05
29     3.383852e-05
           ...     
332    1.702795e-07
333    1.186602e-07
334    2.265309e-07
335    6.135875e-08
336    3.199509e-07
337    1.689704e-07
338    5.439506e-07
339    2.170632e-07
340    1.895438e-08
341    1.152457e-08
342    0.000000e+00
343    2.044615e-10
344    5.060840e-06
345    6.885396e-06
346    4.079250e-03
347    1.758662e-06
348    2.972545e-05
349    1.149698e-05
350    1.211160e-07


In [276]:
test_scaled = merged_df['custom_index_scaled']

In [289]:
def success_of_ico(df):
    
    for i in range(len(df)) : 
        # get the longevity of the ICO
        if df.iloc[i] == 1:
            print('gotcha')
        else:
            continue 

In [293]:
success_of_ico(list)

gotcha


In [14]:
file_path = Path("../../../../data/cleandata/sentiment_cmc_track_merge.csv")
merged_inner.to_csv(file_path, index=False)